In [19]:
#importing all important libraries
import pandas as pd
import os
import sklearn
import numpy as np
import pandas as pd
from sklearn.ensemble import xgboost as xgb
import numpy as np, pandas as pd
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold,GridSearchCV
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
#fifa data set to be used
training_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_21.csv')
testing_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_22.csv')

<ipython-input-20-84a6c8423e0d>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  testing_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_22.csv')


## **1. Data Preparation and Feature Extraction**

Training Data preprocessing

In [21]:
# These are columns that contain ids and url links.
training_data.drop('sofifa_id', axis = 1, inplace = True)
training_data.drop('player_url', axis = 1, inplace = True)
training_data.drop('player_face_url', axis = 1, inplace = True)
training_data.drop('club_logo_url', axis = 1, inplace = True)
training_data.drop('club_flag_url', axis = 1, inplace = True)
training_data.drop('nation_logo_url', axis = 1, inplace = True)
training_data.drop('nation_flag_url', axis = 1, inplace = True)
training_data.drop('nationality_id', axis = 1, inplace = True)
training_data.drop('nation_team_id', axis = 1, inplace = True)
training_data.drop('club_team_id', axis = 1, inplace = True)
training_data.drop('dob', axis = 1, inplace = True) #dropped dob because it basically does the same work as age

In [22]:
# drop columns with 30% or more missing data
missing = (training_data.isnull().sum() / len(training_data)) * 100
columns_missing = missing[missing >= 30].index
training_data.drop(columns=columns_missing, inplace =True)

In [23]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 93 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   short_name                   18944 non-null  object 
 1   long_name                    18944 non-null  object 
 2   player_positions             18944 non-null  object 
 3   overall                      18944 non-null  int64  
 4   potential                    18944 non-null  int64  
 5   value_eur                    18707 non-null  float64
 6   wage_eur                     18719 non-null  float64
 7   age                          18944 non-null  int64  
 8   height_cm                    18944 non-null  int64  
 9   weight_kg                    18944 non-null  int64  
 10  club_name                    18719 non-null  object 
 11  league_name                  18719 non-null  object 
 12  league_level                 18719 non-null  float64
 13  club_position   

In [24]:
Y = pd.DataFrame()
Y = training_data['overall'] # separate the overall column as the Y
training_data.drop('overall', axis = 1, inplace = True) # drop overall from the training data
Y

0        93
1        92
2        91
3        91
4        91
         ..
18939    47
18940    47
18941    47
18942    47
18943    47
Name: overall, Length: 18944, dtype: int64

In [25]:
#grouped columns that are numeric
numeric_training = training_data.select_dtypes(include=['int', 'float']).columns # extract numeric data
numeric_training = training_data[numeric_training]
numeric_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   potential                    18944 non-null  int64  
 1   value_eur                    18707 non-null  float64
 2   wage_eur                     18719 non-null  float64
 3   age                          18944 non-null  int64  
 4   height_cm                    18944 non-null  int64  
 5   weight_kg                    18944 non-null  int64  
 6   league_level                 18719 non-null  float64
 7   club_jersey_number           18719 non-null  float64
 8   club_contract_valid_until    18719 non-null  float64
 9   weak_foot                    18944 non-null  int64  
 10  skill_moves                  18944 non-null  int64  
 11  international_reputation     18944 non-null  int64  
 12  release_clause_eur           17949 non-null  float64
 13  pace            

In [26]:
#impute the numeric data using median
imp = SimpleImputer(strategy = 'median')
numeric_training = pd.DataFrame(imp.fit_transform(numeric_training), columns=numeric_training.columns)
numeric_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   potential                    18944 non-null  float64
 1   value_eur                    18944 non-null  float64
 2   wage_eur                     18944 non-null  float64
 3   age                          18944 non-null  float64
 4   height_cm                    18944 non-null  float64
 5   weight_kg                    18944 non-null  float64
 6   league_level                 18944 non-null  float64
 7   club_jersey_number           18944 non-null  float64
 8   club_contract_valid_until    18944 non-null  float64
 9   weak_foot                    18944 non-null  float64
 10  skill_moves                  18944 non-null  float64
 11  international_reputation     18944 non-null  float64
 12  release_clause_eur           18944 non-null  float64
 13  pace            

In [27]:
#grouped all columns with non-numeric values and are objects

object_training = training_data.select_dtypes(exclude= ['int', 'float']).columns # extract the object data
object_training = training_data[object_training]
object_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   short_name        18944 non-null  object
 1   long_name         18944 non-null  object
 2   player_positions  18944 non-null  object
 3   club_name         18719 non-null  object
 4   league_name       18719 non-null  object
 5   club_position     18719 non-null  object
 6   club_joined       17961 non-null  object
 7   nationality_name  18944 non-null  object
 8   preferred_foot    18944 non-null  object
 9   work_rate         18944 non-null  object
 10  body_type         18944 non-null  object
 11  real_face         18944 non-null  object
 12  ls                18944 non-null  object
 13  st                18944 non-null  object
 14  rs                18944 non-null  object
 15  lw                18944 non-null  object
 16  lf                18944 non-null  object
 17  cf          

In [28]:
#imputed the non numeric values using most_frequent ie. to fill in with the most appearing
imp = SimpleImputer(strategy='most_frequent') # impute the object data
object_training = pd.DataFrame(imp.fit_transform(object_training), columns = object_training.columns)
object_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   short_name        18944 non-null  object
 1   long_name         18944 non-null  object
 2   player_positions  18944 non-null  object
 3   club_name         18944 non-null  object
 4   league_name       18944 non-null  object
 5   club_position     18944 non-null  object
 6   club_joined       18944 non-null  object
 7   nationality_name  18944 non-null  object
 8   preferred_foot    18944 non-null  object
 9   work_rate         18944 non-null  object
 10  body_type         18944 non-null  object
 11  real_face         18944 non-null  object
 12  ls                18944 non-null  object
 13  st                18944 non-null  object
 14  rs                18944 non-null  object
 15  lw                18944 non-null  object
 16  lf                18944 non-null  object
 17  cf          

In [29]:
#encode the object data using the label encoder
encoded_object = pd.DataFrame() # encode the object data
for c in object_training.columns:
  label_encoder = LabelEncoder()
  encoded_object[c] = label_encoder.fit_transform(object_training[c])
encoded_object.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   short_name        18944 non-null  int64
 1   long_name         18944 non-null  int64
 2   player_positions  18944 non-null  int64
 3   club_name         18944 non-null  int64
 4   league_name       18944 non-null  int64
 5   club_position     18944 non-null  int64
 6   club_joined       18944 non-null  int64
 7   nationality_name  18944 non-null  int64
 8   preferred_foot    18944 non-null  int64
 9   work_rate         18944 non-null  int64
 10  body_type         18944 non-null  int64
 11  real_face         18944 non-null  int64
 12  ls                18944 non-null  int64
 13  st                18944 non-null  int64
 14  rs                18944 non-null  int64
 15  lw                18944 non-null  int64
 16  lf                18944 non-null  int64
 17  cf                18944 non-nul

In [30]:
# combine numeric column and object column with overall
training_data = pd.DataFrame()
training_data = pd.concat([encoded_object, numeric_training], axis = 1)
training_data['overall'] = Y
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 93 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   short_name                   18944 non-null  int64  
 1   long_name                    18944 non-null  int64  
 2   player_positions             18944 non-null  int64  
 3   club_name                    18944 non-null  int64  
 4   league_name                  18944 non-null  int64  
 5   club_position                18944 non-null  int64  
 6   club_joined                  18944 non-null  int64  
 7   nationality_name             18944 non-null  int64  
 8   preferred_foot               18944 non-null  int64  
 9   work_rate                    18944 non-null  int64  
 10  body_type                    18944 non-null  int64  
 11  real_face                    18944 non-null  int64  
 12  ls                           18944 non-null  int64  
 13  st              

# **2. Feature Subsets**

In [31]:
#removing overall from the training data
Y = training_data['overall'] # extract overall
training_data.drop('overall', axis = 1, inplace = True) # drop overall
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 92 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   short_name                   18944 non-null  int64  
 1   long_name                    18944 non-null  int64  
 2   player_positions             18944 non-null  int64  
 3   club_name                    18944 non-null  int64  
 4   league_name                  18944 non-null  int64  
 5   club_position                18944 non-null  int64  
 6   club_joined                  18944 non-null  int64  
 7   nationality_name             18944 non-null  int64  
 8   preferred_foot               18944 non-null  int64  
 9   work_rate                    18944 non-null  int64  
 10  body_type                    18944 non-null  int64  
 11  real_face                    18944 non-null  int64  
 12  ls                           18944 non-null  int64  
 13  st              

In [32]:
#scaling the data
sc = StandardScaler()  # scale the data set to be used in the model
training_x = sc.fit_transform(training_data)
training_x = pd.DataFrame(training_x, columns = training_data.columns)
training_x.head(5)

,short_name,long_name,player_positions,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,0.223634,0.153247,1.568777,-0.491360,1.206900,-2.570374,-2.830054,-1.506000,-1.79263,0.439865,...,1.719677,3.137573,-0.717436,-0.587881,-1.028304,-0.594307,-0.310862,-0.066793,-0.130810,-0.478324
1,-1.096741,-1.122359,1.831158,0.126154,0.002573,-0.984269,-0.287383,0.907801,0.55784,-1.649167,...,2.293975,3.055051,-0.915760,-0.728055,-1.028304,-0.537414,-0.310862,-0.066793,-0.130810,-0.310517
2,1.052997,0.955762,1.683569,-0.481319,-0.351641,0.723843,-2.196078,0.887170,0.55784,-1.300995,...,2.549219,2.477402,-0.568694,-0.260807,-1.266928,-0.082270,-0.607686,-0.248403,-0.483401,-0.366452
3,0.796225,0.640429,0.431791,0.628198,-0.493327,-0.862261,-0.844650,-1.217169,0.55784,-1.300995,...,2.804462,2.890009,-0.568694,-0.821505,-0.789680,-0.423628,-0.429592,-0.066793,-0.072045,-0.310517
4,0.015420,0.029530,-1.322884,0.357095,-0.635012,-0.252221,-1.749363,-1.340954,0.55784,-1.997338,...,2.293975,2.724966,1.067476,0.813864,0.355717,-0.082270,-0.192133,-0.672158,-0.365871,-0.198645


In [33]:
#Training using rf for Feature Importance
model=RandomForestRegressor()
model.fit(training_x,Y)

RandomForestRegressor()

In [34]:
#Getting the Important Features from our dataset
name_of_feature=training_x.columns #contains the names of features in our dataset
feature_importance=model.feature_importances_ #contains the scores of each feature

#Sorting the Feature Importance
feature_importance_df=pd.DataFrame({'Feature':name_of_feature,'Importance':feature_importance}) #creates a dataframe with the names of our feature and corresponding scores
feature_importance_df=feature_importance_df.sort_values(by='Importance',ascending=False) #sorts our results from highest to lowest
feature_importance_df



,Feature,Importance
40,value_eur,0.471493
51,release_clause_eur,0.316380
42,age,0.116189
39,potential,0.053675
71,movement_reactions,0.028395
...,...,...
45,league_level,0.000019
48,weak_foot,0.000018
49,skill_moves,0.000010
8,preferred_foot,0.000007


In [41]:
#Using the first 10 features
first_10_features = feature_importance_df['Feature'].values[:11]
first_10_features

array(['value_eur', 'release_clause_eur', 'age', 'potential',
       'movement_reactions', 'wage_eur', 'gk', 'lm', 'rm', 'lcb', 'cf'],
      dtype=object)

In [44]:
#updates to our new training_x value
training_x=training_x[first_10_features]


# **3. Machine learning model with cross validation**

In [45]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(training_x,Y,test_size=0.2,random_state=42) #splitting our data

In [46]:
dt = DecisionTreeRegressor(criterion ='squared_error', random_state = 42)
gb = GradientBoostingRegressor(init = dt, n_estimators = 200, learning_rate = 0.01)
XGB_model = xgb.XGBRegressor(objective="reg:squarederror")
rf=RandomForestRegressor(n_estimators=1000, n_jobs = -1)

In [47]:
# Gradient Boosting Regressor
gb.fit(Xtrain,Ytrain)

GradientBoostingRegressor(init=DecisionTreeRegressor(random_state=42),
                          learning_rate=0.01, n_estimators=200)

In [48]:
# Cross validation with the gradient boosting regresssor
cv=KFold(n_splits=5)
PARAMETERS ={
"max_depth":[5,10, 12],
"n_estimators":[50,70,100],
"max_features":['sqrt', 'log2'],
"criterion":['squared_error', 'friedman_mse'],
"random_state": [20, 22, 24]}
gb_cv = GridSearchCV(estimator = gb,param_grid=PARAMETERS,cv=cv,scoring="neg_mean_absolute_error")
gb_cv.fit(Xtrain, Ytrain)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=GradientBoostingRegressor(init=DecisionTreeRegressor(random_state=42),
                                                 learning_rate=0.01,
                                                 n_estimators=200),
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [5, 10, 12],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 70, 100],
                         'random_state': [20, 22, 24]},
             scoring='neg_mean_absolute_error')

In [49]:
best_parameters = gb_cv.best_params_
print("Best parameters:", best_parameters) # display the best parameters for the gradient boosting regressor

Best parameters: {'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 50, 'random_state': 20}


In [50]:
y_pred=gb_cv.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

The mean absolute error is  0.2301398785959356


In [51]:
# Random Forest Regressor
rf.fit(Xtrain, Ytrain)

RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [52]:
# Cross validation with the random forest regresssor
cv=KFold(n_splits=5)
PARAMETERS ={
"max_depth": [10, 12, 15],
"n_estimators": [50, 70, 100],
"criterion": ['squared_error', 'friedman_mse'],
"max_features": ['sqrt', 'log2'],
"random_state": [20, 22, 24]}
rf_cv = GridSearchCV(estimator = rf,param_grid=PARAMETERS,cv=cv,scoring="neg_mean_absolute_error")
rf_cv.fit(Xtrain, Ytrain)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestRegressor(n_estimators=1000, n_jobs=-1),
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [10, 12, 15],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 70, 100],
                         'random_state': [20, 22, 24]},
             scoring='neg_mean_absolute_error')

In [53]:
best_parameters = rf_cv.best_params_
print("Best parameters:", best_parameters) # display the best parameters for the random forest regressor

Best parameters: {'criterion': 'squared_error', 'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 100, 'random_state': 24}


In [54]:
y_pred=rf_cv.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

The mean absolute error is  0.3540056436657964


In [55]:
# Cross validation with the XGB regresssor
cv=KFold(n_splits=5)
PARAMETERS ={
"max_depth": [10, 12, 15],
"random_state": [20, 22, 24]}
xgb_cv = GridSearchCV(estimator = XGB_model,param_grid=PARAMETERS,cv=cv,scoring="neg_mean_absolute_error")
xgb_cv.fit(Xtrain, Ytrain)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    i...
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'max_depth': [10, 12, 15],
                         'random_state': [20, 22, 24]},
             scoring='neg_mean_absolute_error')

In [56]:
best_parameters = xgb_cv.best_params_
print("Best parameters:", best_parameters) # display the best parameters for the XGB regressor

Best parameters: {'max_depth': 15, 'random_state': 20}


In [57]:
y_pred=xgb_cv.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

The mean absolute error is  0.23103051224620716


# **4. Model Optimization**
Fine tuning , training and re-testing



In [59]:
training_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   value_eur           18944 non-null  float64
 1   release_clause_eur  18944 non-null  float64
 2   age                 18944 non-null  float64
 3   potential           18944 non-null  float64
 4   movement_reactions  18944 non-null  float64
 5   wage_eur            18944 non-null  float64
 6   gk                  18944 non-null  float64
 7   lm                  18944 non-null  float64
 8   rm                  18944 non-null  float64
 9   lcb                 18944 non-null  float64
 10  cf                  18944 non-null  float64
dtypes: float64(11)
memory usage: 1.6 MB


In [61]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(training_x,Y,test_size=0.2,random_state=42) #splitting our data

In [64]:
#cross validation with gb boost which was our best performing model
cv=KFold(n_splits=5)
PARAMETERS ={
"max_depth":[7,10,15],
"n_estimators":[50,70,120],
"max_features":['sqrt', 'log2'],
"criterion":['squared_error', 'friedman_mse'],
"random_state": [20, 22, 24]}
gb_cv = GridSearchCV(estimator = gb,param_grid=PARAMETERS,cv=cv,scoring="neg_mean_absolute_error")
gb_cv.fit(Xtrain, Ytrain)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=GradientBoostingRegressor(init=DecisionTreeRegressor(random_state=42),
                                                 learning_rate=0.01,
                                                 n_estimators=200),
             param_grid={'criterion': ['squared_error', 'friedman_mse'],
                         'max_depth': [7, 10, 15],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 70, 120],
                         'random_state': [20, 22, 24]},
             scoring='neg_mean_absolute_error')

In [65]:
#display the best parameters for the GB regressor
best_parameters = gb_cv.best_params_
print("Best parameters:", best_parameters)

Best parameters: {'criterion': 'squared_error', 'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 50, 'random_state': 20}


In [67]:
y_pred=gb_cv.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

The mean absolute error is  0.0005278437582475588


# **5. Model testing with player 22 data**

In [68]:
#using specific columns used in our training data in our testing data
new_testing_data = testing_data[training_x.columns]
new_testing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   value_eur           19165 non-null  float64
 1   release_clause_eur  18063 non-null  float64
 2   age                 19239 non-null  int64  
 3   potential           19239 non-null  int64  
 4   movement_reactions  19239 non-null  int64  
 5   wage_eur            19178 non-null  float64
 6   gk                  19239 non-null  object 
 7   lm                  19239 non-null  object 
 8   rm                  19239 non-null  object 
 9   lcb                 19239 non-null  object 
 10  cf                  19239 non-null  object 
 11  overall             19239 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 1.8+ MB


In [69]:
Y = testing_data['overall']
Y

0        93
1        92
2        91
3        91
4        91
         ..
19234    47
19235    47
19236    47
19237    47
19238    47
Name: overall, Length: 19239, dtype: int64

In [70]:
#grouped columns that are numeric
numeric_testing = new_testing_data.select_dtypes(include=['int', 'float']).columns
numeric_testing = new_testing_data[numeric_testing]
numeric_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   value_eur           19165 non-null  float64
 1   release_clause_eur  18063 non-null  float64
 2   age                 19239 non-null  int64  
 3   potential           19239 non-null  int64  
 4   movement_reactions  19239 non-null  int64  
 5   wage_eur            19178 non-null  float64
 6   overall             19239 non-null  int64  
dtypes: float64(3), int64(4)
memory usage: 1.0 MB


In [71]:
#grouped all columns with non-numeric values and object
object_testing = new_testing_data.select_dtypes(exclude=['int', 'float']).columns
object_testing = new_testing_data[object_testing]
object_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gk      19239 non-null  object
 1   lm      19239 non-null  object
 2   rm      19239 non-null  object
 3   lcb     19239 non-null  object
 4   cf      19239 non-null  object
dtypes: object(5)
memory usage: 751.6+ KB


In [72]:
#imputed the numeric values using most_frequent ie. to fill in with the mean
imp = SimpleImputer(strategy='mean')
numeric_testing = pd.DataFrame(imp.fit_transform(numeric_testing), columns = numeric_testing.columns)
numeric_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   value_eur           19239 non-null  float64
 1   release_clause_eur  19239 non-null  float64
 2   age                 19239 non-null  float64
 3   potential           19239 non-null  float64
 4   movement_reactions  19239 non-null  float64
 5   wage_eur            19239 non-null  float64
 6   overall             19239 non-null  float64
dtypes: float64(7)
memory usage: 1.0 MB


In [73]:
#encode the object data using the label encoder
encoded_object = pd.DataFrame()
for c in object_testing.columns:
  label_encoder = LabelEncoder()
  encoded_object[c] = label_encoder.fit_transform(object_testing[c])
encoded_object.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   gk      19239 non-null  int64
 1   lm      19239 non-null  int64
 2   rm      19239 non-null  int64
 3   lcb     19239 non-null  int64
 4   cf      19239 non-null  int64
dtypes: int64(5)
memory usage: 751.6 KB


In [74]:
#rejoined the numeric and non-numeric columns
testing_x = pd.DataFrame()
testing_x = pd.concat([encoded_object, numeric_testing], axis = 1)
testing_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gk                  19239 non-null  int64  
 1   lm                  19239 non-null  int64  
 2   rm                  19239 non-null  int64  
 3   lcb                 19239 non-null  int64  
 4   cf                  19239 non-null  int64  
 5   value_eur           19239 non-null  float64
 6   release_clause_eur  19239 non-null  float64
 7   age                 19239 non-null  float64
 8   potential           19239 non-null  float64
 9   movement_reactions  19239 non-null  float64
 10  wage_eur            19239 non-null  float64
 11  overall             19239 non-null  float64
dtypes: float64(7), int64(5)
memory usage: 1.8 MB


In [75]:
sc = StandardScaler()
new_testing_x = sc.fit_transform(testing_x)
new_testing_x = pd.DataFrame(new_testing_x, columns = testing_x.columns)
new_testing_x.head(5)

,gk,lm,rm,lcb,cf,value_eur,release_clause_eur,age,potential,movement_reactions,wage_eur,overall
0,0.068706,3.052455,3.052455,-0.533389,3.210690,9.889601,9.591752,1.851089,3.60178,3.599846,15.998022,3.957501
1,0.068706,2.843314,2.843314,0.334698,3.090926,15.350958,13.244084,1.429869,3.43747,3.489252,13.425844,3.812154
2,0.184414,2.968799,2.968799,-0.320796,3.130847,5.546836,5.380179,2.272309,3.27316,3.599846,13.425844,3.666806
3,0.184414,3.010627,3.010627,-0.533389,3.090926,16.601147,16.109335,0.798039,3.27316,3.046874,13.425844,3.666806
4,0.300122,3.010627,3.010627,1.379946,3.051005,16.140551,15.660560,1.008649,3.27316,3.268063,17.541329,3.666806


In [76]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(testing_x,Y,test_size=0.2,random_state=42)#split data

In [77]:
gb = GradientBoostingRegressor(n_estimators = 50, random_state = 20) #using the best perfomind model to train our 2022 data

In [78]:
gb.fit(Xtrain, Ytrain)

GradientBoostingRegressor(n_estimators=50, random_state=20)

In [80]:
y_pred = gb.predict(Xtest)
print("The mean absolute error is ",(mean_absolute_error(y_pred,Ytest))) # display the mean absolute error

The mean absolute error is  0.03281081265922101


# 6. Saving the model using pickle

In [82]:
import pickle

In [94]:
deployment_model = '/content/drive/MyDrive/Colab Notebooks/deployment_model.pkl'
pickle.dump(gb, open(deployment_model, 'wb'))
loaded_model = pickle.load(open(deployment_model, 'rb'))

In [99]:
scaler_model = '/content/drive/MyDrive/Colab Notebooks/scalar_model.pkl'
pickle.dump(sc,open(scaler_model, 'wb'))
loaded_model = pickle.load(open(scaler_model, 'rb'))

In [108]:
try:
  testing_values = {'Ytest': Ytest, 'y_pred': y_pred}
  testing_values_df = pd.DataFrame(testing_values)

  testing_values_df.to_csv("/content/drive/MyDrive/Colab Notebooks/Testing_values.csv", index=False)
  print("Testing values saved to Testing_values.csv successfully.")
except Exception as e:
  print("An error occurred while saving the testing values:", str(e))

Testing values saved to Testing_values.csv successfully.
